# Text Extraction

NOTES

- The primary key found was endpoint and controller
- The number of requests and responses mismatched. The removed user was chosen was based on the request with the same primary key and only one response. The specific user removed is based on their 0% new year resolution, and as such would likely not have a need for the data visualizations.

## Imports

In [66]:
import re
import json
import pandas as pd

## Data Extraction

### All Data

In [76]:
items = []

# open file
with open("data_cleaned.txt") as f:
    lines = [line.rstrip() for line in f] # read all files and remove whitespace
    for line in lines:
        if re.search("Request|Response", line): # Each request/response makes a new dict
            items.append({})
        elif len(line) == 0: 
            pass # skip if there is no content in the line
        else:
            k, v = re.split(":\s", line) # add the key value pair to the list
            items[-1][k.strip()] = v.strip() # Add to dictionary and remove surrounding whitespace

### Extract Discipline

In [85]:
for i in items:
    i['Discipline'] = re.search("edge\/(.*)$", i['Endpoint']).group(1) # Extract the displine from endpoint using regex

### Match Request and Responses

In [93]:
users = [] # list of users
found = set() # set of found users using Primary Key of discipline and Controller
for i in range(len(items)):
    i1 = items[i]
    if (i1['Controller'], i1['Discipline']) in found: # if the user already exists, skip
        continue
    for j in range(i+1, len(items)):
        i2 = items[j]
        if i1['Controller'] == i2['Controller'] and i1['Discipline'] == i2['Discipline']: # if the primary keys aforementioned match
            users.append({}) # make a new user
            for k, v in i1.items():
                users[-1][k] = v # add all keys and values to user
            for k, v in i2.items():
                users[-1][k] = v # add all keys and values to user
            found.add((i1['Controller'], i1['Discipline']))
            break

### Value Checks

In [100]:
print(
    len(users), users[0]
)

15 {'Address': '178373', 'Red Meat': '100 pounds', 'Controller': 'LocationController', 'Grains': '2 pounds', 'Action': 'Get', 'Endpoint': '(OWL) https://uwo.ca/edge/software', 'Dairy': '200 pounds', 'Cellphone': '55 hours', 'TV': '43 hours', 'Computer': '130 hours', 'New Year Resolution': '5%', 'Discipline': 'software', 'Username': 'Sammy', 'Car': '150 hours', 'Walking': '3 hours', 'Public Transport': '10 hours', 'Status': '4', 'Items/Total': '2/1', 'Red Meat CO2 Emissions': 800.0, 'Grains CO2 Emissions': 7.4, 'Dairy CO2 Emissions': 1260.0, 'Cellphone CO2 Emissions': 198.0, 'TV CO2 Emissions': 292.4, 'Computer CO2 Emissions': 546.0, 'Car CO2 Emissions': 975.0, 'Walking CO2 Emissions': 0, 'Public Transport CO2 Emissions': 43.0}


### Add Conversion Metrics

In [106]:
conversions = {} 
with open('conversion_table.json') as f:
    conversions = json.loads(f.read()) # Using the data provided in the product requirements
conversions

{'Public Transport': 4.3,
 'Car': 6.5,
 'Walking': 0,
 'Red Meat': 8.0,
 'Dairy': 6.3,
 'Grains': 3.7,
 'Cellphone': 3.6,
 'Computer': 4.2,
 'TV': 6.8}

In [115]:
for u in users:
    for k in ['Red Meat', 'Grains', 'Dairy', 'Cellphone', 'TV', 'Computer', 'Car', 'Walking', 'Public Transport']: # List of conversion factors from the text file
        units = int(re.search("^(.*)\s", u[k]).group(1)) # extract of units and converted into integers
        CO2 = units * conversions[k] # mutliply units by conversion factors
        col_name = f"{k} CO2 Emissions"
        u[col_name] = CO2 # Add new column to table

## Extract CSV

### JSON To CSV Conversion

In [119]:
df = pd.DataFrame(users)
df.head()

,Address,Red Meat,Controller,Grains,Action,Endpoint,Dairy,Cellphone,TV,Computer,...,Items/Total,Red Meat CO2 Emissions,Grains CO2 Emissions,Dairy CO2 Emissions,Cellphone CO2 Emissions,TV CO2 Emissions,Computer CO2 Emissions,Car CO2 Emissions,Walking CO2 Emissions,Public Transport CO2 Emissions
0,178373,100 pounds,LocationController,2 pounds,Get,(OWL) https://uwo.ca/edge/software,200 pounds,55 hours,43 hours,130 hours,...,2/1,800.0,7.4,1260.0,198.0,292.4,546.0,975.0,0,43.0
1,156354,89 pounds,Controlmite,167 pounds,Get,(OWL) https://uwo.ca/edge/software,42 pounds,42 hours,75 hours,88 hours,...,3/1,712.0,617.9,264.6,151.2,510.0,369.6,487.5,0,184.9
2,919323,26 pounds,ControlPublic,78 pounds,Get,https://uwo.ca/edge/electrical,78 pounds,100 hours,50 hours,75 hours,...,3/5,208.0,288.6,491.4,360.0,340.0,315.0,546.0,0,335.4
3,983575,35 pounds,Controlz,78 pounds,Get,(OWL) https://uwo.ca/edge/mechanical,64 pounds,78 hours,50 hours,5 hours,...,3/1,280.0,288.6,403.2,280.8,340.0,21.0,130.0,0,146.2
4,919325,55 pounds,LocationController,44 pounds,Get,https://uwo.ca/edge/mechanical,88 pounds,0 hours,100 hours,50 hours,...,3/3,440.0,162.8,554.4,0.0,680.0,210.0,507.0,0,94.6


### Rename Columns

In [122]:
new_columns = []
for col in df.columns:
    new_columns.append(col.lower().replace(" ", "_")) # Make all columns standardized formats
df.columns = new_columns

### Extract to CSV

In [123]:
df.to_csv("emissions.csv")